## dependencies installeren

In [23]:
import sys
if 'installatie_van_dependencies_is_voltooid' not in globals():
    print('Python',sys.version)
    try:
        import pydal
    except: 
        print("Heel veel packages aan het installeren")
        !pip install psycopg2-binary pydal sshtunnel ipython-sql sqlalchemy traitlets ycecream more-itertools jupyterlab_myst python-dotenv> /dev/null

    try:
        import backend
    except:
        print("Backend installeren")
        !pip install -i https://devpi.edwh.nl/remco/dev/+simple backend==${VERSION} > /dev/null 
        # traitlets moet wel de laatste versie zijn, hoewel dit mogelijk niet goed gaat ergens in de dependencies. 
        # maar als traitlets niet de laatste versie is, zijn er dependencies van jupyterlab die stuk gaan. 
        # niethandig. 
        print("jupyterlab niet stuk maken")
        !pip install traitlets --upgrade  > /dev/null
        # volgende werkt niet want heeft ook een jupyter optie nodig om chromium te mogen
        # installeren. 
        # # installeer onderstaande voor webpdf support.
        # !pip install nbconvert[webpdf] > /dev/null 
        global installatie_van_dependencies_is_voltooid
        installatie_van_dependencies_is_voltooid = True 
else: 
    print('Python', sys.version)
    print('Dependencies reeds geinstalleerd')

# https://github.com/theskumar/python-dotenv
from dotenv import load_dotenv
load_dotenv('.env')
import os
# gebruik secrets via os.getenv()

Python 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:20:46) 
[GCC 9.4.0]


In [2]:
from IPython.display import display, Markdown

## libs importeren om verbinding op te bouwen 

In [3]:
from ycecream import y 

In [4]:
from icecream import ic

In [5]:
import os, pydal
import random
import uuid
import numpy as np
from bs4 import BeautifulSoup
from icecream import ic
import ycecream # ycecream werkt niet in jupyterbooks :(
from tqdm import tqdm
import hashlib
import pandas as pd
import more_itertools
from collections import Counter
from pathlib import Path
from contextlib import contextmanager
import slugify 
from nameko.standalone.events import event_dispatcher
# from nameko.standalone.rpc import ClusterRpcProxy # error met protocl udp niet gevonden?

# stel een standaard label in. per iteratie eigenlijk opnieuw doen. 
tqdm.pandas(desc='Jupyter')

# print('De weg naar postgres gevonden:', 'POSTGRES_URI' in os.environ)
from backend import define_tables
try:
    # try to silently close any open databases from a previous run 
    db.close()
except:
    pass 
db = pydal.DAL(os.getenv('POSTGRES_URI'), migrate=False )
db.executesql("PGPOOL SET client_idle_limit = 3600;")
define_tables.setup_db_tables(db, migrate=False)
# rows = db((db.item.author!=None) & (db.item.platform.contains('SvS'))).select(db.item.gid, db.item.name, limitby=(0,10), orderby=~db.item.ts_changed)
# print()
# for table in sorted(db.tables):
#     print(f'# {table}')
#     print(f'  ',', '.join(db[table].fields))
#     print('')
# rows.as_csv().split('\r\n')

<DAL uri="postgres://******@pgpool:5432/backend">

## SQL toegang krijgen middels 
zodat dit kan werken: 
```
%%sql
select * from "user" limit (10)
```
of bijvoorbeeld 
```
%%sql
select to_char(ts,'YYYY-MM') as year_month,  count(*) as  items_created
from event_stream
where platform = 'SvS'
  and subject_type = 'item'
  and crud = 'c'
group by crud, subject_type,  to_char(ts,'YYYY-MM')
order by to_char(ts,'YYYY-MM') desc , subject_type, crud;
```

In [6]:
%load_ext sql
DB_URI=db._uri.replace('postgres://','postgresql://')
%sql $DB_URI
del DB_URI

In [7]:
%%sql
-- zorg er voor dat deze verbinding niet na 30 seconden gesloten wordt 
PGPOOL SET client_idle_limit = 3600;

 * postgresql://postgres@pgpool:5432/backend
Done.


[]

## Panda's gebruiken vanaf pydal resultaat

In [8]:
import pandas as pd
from io import BytesIO
#items = pd.read_csv(BytesIO(rows.as_csv().encode('utf-8')))

## Gebruik van libraries icm jupyterlab 

In [9]:
import handig
handig.zeer_handig

True

In [10]:
# als de library gewijzigd is en je wil niet de hele kernel herstarten
# maar wel de nieuwe library gebruiken, kun je dit doen: 
from importlib import reload 
handig = reload(handig)

## Tags


In [11]:
Tag = handig.TagFactory(db)

# make available as regular variables in the python_environment
Tag.create_Tnamed_metatag_variables(locals())

# # show these items
# for k,v in locals().copy().items():
#     if k.startswith('T') and isinstance(v,Tag):
#         print(f'{k:>20} : {v.gid}')

In [12]:
# find all filters :
filter_tags = Tag.find_by_meta_tag(TFilter)
# dict([(r.name, Tag.find_children(Tag(r))) for r in filter_tags])

# Zoek fouten in de Tag structuur

Een parent property en een child property hoeven niet in sync met elkaar te zijn. Dus door het vergelijken van de parents van elke child, en de children van elke parent is na te gaan waar deze niet langer kloppend zijn.  
<font color="red"> **⚠** Fix deze issues </font>

In [13]:
def _print(tag, parent, depth):
    if tag == parent:
        return False
    print('   ' * depth, tag.name, handig.names(tag.metatags()))
    if tag.name == 'User generated tags':
        print('   ' * (depth + 1), ' *** hidden ***')
    return tag.name != 'User generated tags'